In [1]:
import sys
sys.path.insert(0,'/Users/marcosaponara/Desktop/MY_CENTIPEDE_GAME/CODE/mysrc')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from json import load

from itertools import product
from tqdm import tqdm

from egttools.analytical import PairwiseComparison
from egttools.games import Matrix2PlayerGameHolder
from egttools.utils import calculate_stationary_distribution
from egttools.plotting import draw_invasion_diagram

from utils.centipedeGame import CentipedeGame
from utils.kStrategy import KStrategy

from utils.kernel import eps_kernel_sym as kernel

In [3]:
from scipy.spatial.distance import jensenshannon

def hellinger(p : np.ndarray, 
              q : np.ndarray,
             ):
    return np.sqrt(np.sum((np.sqrt(p) - np.sqrt(q)) ** 2, axis = -1)) / np.sqrt(2)

In [4]:
with open('/Users/marcosaponara/Desktop/MY_CENTIPEDE_GAME/CODE/mysrc/data/data.json') as f:
    data = load(f)

In [5]:
GAMEID = 'icg6'

In [6]:
icgdata = data['data'][GAMEID]

nb_steps = icgdata['nb_steps']
payoffs_pl1 = np.array(icgdata['payoffs_pl1'], dtype = float)
payoffs_pl2 = np.array(icgdata['payoffs_pl2'], dtype = float)
experimental_reference = np.array(icgdata['experimental_reference'], dtype = float)

In [7]:
nb_k_levels = nb_steps + 1
k_levels = np.arange(nb_k_levels)

In [8]:
start = np.zeros(nb_steps+2, dtype = float)

start[-1] = 1.
start[nb_steps//2] = 1.

start_arrays = np.array([
                        start,
                        ], dtype = float)

start_k_pairs = list(product(start_arrays, k_levels))
nb_strategies = len(start_k_pairs)

In [9]:
# Parameters
Z = 1000 # population size

nb_beta_datapoints = 101
betas = np.logspace(-4., 2., nb_beta_datapoints)

gamma = 0.

if GAMEID=='icg4':
    eps = .09
elif GAMEID=='icg6':
    eps = .045

In [10]:
ker = kernel(nb_steps, gamma, eps)

strategy = KStrategy(ker)

strategies = []
for pair in start_k_pairs:
    start, k = pair
    strategies.append(strategy.calculate_mixed_strategy(k = k, start = start))
strategies = np.array(strategies)
nb_strategies = len(strategies)

cg = CentipedeGame(payoffs_pl1, payoffs_pl2, strategies)
game = Matrix2PlayerGameHolder(nb_strategies, cg.payoffs())
evolver = PairwiseComparison(population_size=Z, game=game)

sd = np.zeros((nb_beta_datapoints, nb_strategies), dtype = float)
avg_take_distribution = np.zeros((nb_beta_datapoints, nb_steps+1), dtype = float)

for i, beta in enumerate(tqdm(betas)):
    transition_matrix, _ = evolver.calculate_transition_and_fixation_matrix_sml(beta)
    sd[i,:] = calculate_stationary_distribution(transition_matrix.transpose())
    avg_take_distribution[i,:] = sd[i,:] @ (sd[i,:] @ cg.get_take_distributions())

rmse = np.sqrt(((avg_take_distribution - experimental_reference) ** 2).sum(axis = -1))
js = jensenshannon(avg_take_distribution, experimental_reference, axis = -1) 
hel = hellinger(avg_take_distribution, experimental_reference)

100%|█████████████████████████████████████████| 101/101 [00:11<00:00,  8.99it/s]


In [11]:
# # saving output data
# np.save('./' + GAMEID + '_sd.npy', sd)
# np.save('./' + GAMEID + '_js.npy', js)
# np.save('./' + GAMEID + '_rmse.npy', rmse)
# np.save('./' + GAMEID + '_hel.npy', hel)
# np.save('./' + GAMEID + '_beta_array.npy', betas)